<a href="https://colab.research.google.com/github/daman1209arora/cnn-mnist/blob/main/mnist_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

# **Classification using a normal neural network**

**Architecture:**

*   4 fully connected layers consisting of 100, 35, 23, and 10 neurons
*   Activation function used is relU for all but the last layer. In that last layer, softmax is used.


In [2]:
class NNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 100)
        self.fc2 = nn.Linear(100, 35)
        self.fc3 = nn.Linear(35, 23)
        self.fc4 = nn.Linear(23, 10)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.softmax(self.fc4(x), dim=1)
        return x
    

nn_model = NNModel()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(nn_model.parameters())


# **Code for setting up the training**

*   Batch size = 50
*   No of epochs = 5
*   Train-test split = 90:10
*   Optimiser = Adam



In [3]:
mnist_data = torchvision.datasets.MNIST("./data", download=True, transform=torchvision.transforms.ToTensor())
size = len(mnist_data)

train_data, test_data = torch.utils.data.random_split(mnist_data, [int(0.9 * size), int(0.1 * size)])

train_data_loader = torch.utils.data.DataLoader(train_data, 50, shuffle=True)

total_loss = 0
for epoch in range(5):
    for i, data in enumerate(train_data_loader, 0):
        images, labels = data
        images = torch.flatten(images, start_dim=1)
        optimizer.zero_grad()

        output = nn_model(images)
        loss = loss_function(output, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() 

    print("After epoch {}, loss: {}".format(epoch + 1, total_loss))
    total_loss = 0

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


After epoch 1, loss: 1823.3163739442825
After epoch 2, loss: 1738.1538286209106
After epoch 3, loss: 1724.8848419189453
After epoch 4, loss: 1706.33795773983
After epoch 5, loss: 1634.0281180143356


#**Evaluating model on Test data**

In [4]:
correct = 0
total = 0

test_data_loader = torch.utils.data.DataLoader(test_data, 5, shuffle=True)
with torch.no_grad():
    for i, data in enumerate(test_data_loader, 0):
        images, labels = data
        images = torch.flatten(images, start_dim=1)
        output = nn_model(images)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
print("Accuracy on the test set:", 100 * correct / total)


Accuracy on the test set: 94.68333333333334


# **Classification using a Convolutional Neural Network**
**Architecture:**\
1) Convolutional layer(input channels: 1, output channels: 10, kernel size: 5)\
2) Max-pooling layer(kernel size: 2)\
3) Convolutional layer(input channels: 10, output channels: 15, kernel size: 5)\
4) Flatten the output channels to form a tensor of dimensions (batch_size, 240)\
5) 3 fully connected layers of 100, 35, and 10 neurons



In [5]:
class CNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=15, kernel_size=5)
        self.fc1 = nn.Linear(15 * 4 * 4, 100)
        self.fc2 = nn.Linear(100, 35)
        self.fc3 = nn.Linear(35, 10)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 15 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return F.softmax(self.fc3(x), dim=1)
    

cnn_model = CNNModel()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters())

# **Code for setting up the training**

*   Batch size = 50
*   No of epochs = 5
*   Train-test split = 90:10
*   Optimiser = Adam


In [6]:
mnist_data = torchvision.datasets.MNIST("./data", download=True, transform=torchvision.transforms.ToTensor())
size = len(mnist_data)

train_data, test_data = torch.utils.data.random_split(mnist_data, [int(0.9 * size), int(0.1 * size)])

train_data_loader = torch.utils.data.DataLoader(train_data, 50, shuffle=True)

total_loss = 0
for epoch in range(5):
    for i, data in enumerate(train_data_loader, 0):
        images, labels = data
        #images = torch.flatten(images, start_dim=1)
        optimizer.zero_grad()

        output = cnn_model(images)
        loss = loss_function(output, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() 

    print("After epoch {}, loss: {}".format(epoch + 1, total_loss))
    total_loss = 0

After epoch 1, loss: 1810.3362410068512
After epoch 2, loss: 1721.5754969120026
After epoch 3, loss: 1712.9356054067612
After epoch 4, loss: 1633.770393371582
After epoch 5, loss: 1604.9865123033524


#**Evaluating model on Test data**

In [7]:
correct = 0
total = 0

test_data_loader = torch.utils.data.DataLoader(test_data, 5, shuffle=True)
with torch.no_grad():
    for i, data in enumerate(test_data_loader, 0):
        images, labels = data
        #images = torch.flatten(images, start_dim=1)
        output = cnn_model(images)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
print("Accuracy on the test set:", 100 * correct / total)

Accuracy on the test set: 97.58333333333333


In [ ]:
import torchvision.models as models 

resnet18 = models.resnet18(pretrained=True)
